# Importing stuffs

In [12]:
import numpy as np
import argparse
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Setting the mode to either train or display and changing the batch size and number of epochs

In [13]:
mode = "train"
batch_size = 32
num_epoch = 50

## Defining data generators

In [15]:
train_dir = 'images/train'
val_dir = 'images/validation'

num_train = 28709
num_val = 7178

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


## Creating the model

In [16]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

## When mode = train, it will train the model. If model = display, it will open up the camera and recognize the emotions

In [18]:
if mode == "train":
    model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])
    model_info = model.fit_generator(
            train_generator,
            steps_per_epoch=num_train // batch_size,
            epochs=num_epoch,
            validation_data=validation_generator,
            validation_steps=num_val // batch_size)
    model.save('32_batch_ori_exported_model_git.h5')
    
# emotions will be displayed on your face from the webcam feed
elif mode == "display":
    model.load_weights('exported_model_git.h5')

    # prevents openCL usage and unnecessary logging messages
    cv2.ocl.setUseOpenCL(False)

    # dictionary which assigns each label an emotion (alphabetical order)
    emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

    # start the webcam feed
    cap = cv2.VideoCapture(0)
    while True:
        # Find haar cascade to draw bounding box around face
        ret, frame = cap.read()
        if not ret:
            break
        facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
            prediction = model.predict(cropped_img)
            maxindex = int(np.argmax(prediction))
            cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('Video', cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Epoch 1/50


/var/folders/f6/mgwjx_cs0bl6g_dbgdp8jfq40000gn/T/ipykernel_91617/1110841621.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_info = model.fit_generator(


897/897 [==============================] - ETA: 0s - loss: 1.7756 - accuracy: 0.2763WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 224 batches). You may need to use the repeat() function when building your dataset.


897/897 [==============================] - 130s 145ms/step - loss: 1.7756 - accuracy: 0.2763 - val_loss: 1.6446 - val_accuracy: 0.3731
Epoch 2/50
897/897 [==============================] - 156s 174ms/step - loss: 1.5770 - accuracy: 0.3908
Epoch 3/50
897/897 [==============================] - 148s 165ms/step - loss: 1.4727 - accuracy: 0.4345
Epoch 4/50
897/897 [==============================] - 206s 230ms/step - loss: 1.3967 - accuracy: 0.4682
Epoch 5/50
897/897 [==============================] - 216s 240ms/step - loss: 1.3317 - accuracy: 0.4944
Epoch 6/50
897/897 [==============================] - 240s 267ms/step - loss: 1.2808 - accuracy: 0.5133
Epoch 7/50
897/897 [==============================] - 243s 271ms/step - loss: 1.2350 - accuracy: 0.5319
Epoch 8/50
897/897 [==============================] - 258s 287ms/step - loss: 1.1921 - accuracy: 0.5503
Epoch 9/50
897/897 [==============================] - 250s 279ms/step - loss: 1.1569 - accuracy: 0.5637
Epoch 10/50
897/897 [============